## Deliverable 2. Create a Customer Travel Destinations Map.

In [6]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import google_api_key

# Configure gmaps API key
gmaps.configure(api_key=google_api_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,64.31,87,100,1.99,overcast clouds
1,1,Butaritari,KI,3.0707,172.7902,81.86,77,17,12.66,light rain
2,2,Wau,SS,7.7029,27.9953,72.72,88,80,3.31,broken clouds
3,3,Ponta Do Sol,PT,32.6667,-17.1000,72.23,73,97,3.56,overcast clouds
4,4,Jamestown,US,42.0970,-79.2353,68.72,82,90,6.91,overcast clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Butaritari,KI,3.0707,172.7902,81.86,77,17,12.66,light rain
6,6,Bengkulu,ID,-3.8004,102.2655,84.56,67,13,9.60,few clouds
9,9,Naze,JP,28.3667,129.4833,83.32,83,37,19.75,scattered clouds
10,10,Big Spring,US,32.2504,-101.4787,81.88,32,1,8.01,clear sky
13,13,Hilo,US,19.7297,-155.0900,78.53,80,90,11.50,light rain
15,15,Victoria,HK,22.2855,114.1577,84.38,87,100,1.99,overcast clouds
18,18,Avarua,CK,-21.2078,-159.7750,77.05,69,40,4.61,scattered clouds
28,28,Puerto Escondido,MX,15.8500,-97.0667,80.40,76,100,3.91,overcast clouds
39,39,Porbandar,IN,21.6422,69.6093,80.58,94,20,4.61,haze
40,40,Arlit,NE,18.7369,7.3853,80.13,14,0,8.72,clear sky


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                174
City                   174
Country                174
Lat                    174
Lng                    174
Max Temp               174
Humidity               174
Cloudiness             174
Wind Speed             174
Current Description    174
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
complete_cities_df = preferred_cities_df.dropna()
complete_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Butaritari,KI,3.0707,172.7902,81.86,77,17,12.66,light rain
6,6,Bengkulu,ID,-3.8004,102.2655,84.56,67,13,9.60,few clouds
9,9,Naze,JP,28.3667,129.4833,83.32,83,37,19.75,scattered clouds
10,10,Big Spring,US,32.2504,-101.4787,81.88,32,1,8.01,clear sky
13,13,Hilo,US,19.7297,-155.0900,78.53,80,90,11.50,light rain
15,15,Victoria,HK,22.2855,114.1577,84.38,87,100,1.99,overcast clouds
18,18,Avarua,CK,-21.2078,-159.7750,77.05,69,40,4.61,scattered clouds
28,28,Puerto Escondido,MX,15.8500,-97.0667,80.40,76,100,3.91,overcast clouds
39,39,Porbandar,IN,21.6422,69.6093,80.58,94,20,4.61,haze
40,40,Arlit,NE,18.7369,7.3853,80.13,14,0,8.72,clear sky


In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = complete_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Butaritari,KI,81.86,light rain,3.0707,172.7902,
6,Bengkulu,ID,84.56,few clouds,-3.8004,102.2655,
9,Naze,JP,83.32,scattered clouds,28.3667,129.4833,
10,Big Spring,US,81.88,clear sky,32.2504,-101.4787,
13,Hilo,US,78.53,light rain,19.7297,-155.0900,
15,Victoria,HK,84.38,overcast clouds,22.2855,114.1577,
18,Avarua,CK,77.05,scattered clouds,-21.2078,-159.7750,
28,Puerto Escondido,MX,80.40,overcast clouds,15.8500,-97.0667,
39,Porbandar,IN,80.58,haze,21.6422,69.6093,
40,Arlit,NE,80.13,clear sky,18.7369,7.3853,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_api_key
}

In [20]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print("Match Found")
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Hotel not found... skipping.
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Hotel not found... skipping.
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Hotel not found... skipping.
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Match Found
Hotel not found... skipping.
Match Found
Match Found
Hotel not found... s

In [21]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Butaritari,KI,81.86,light rain,3.0707,172.7902,Isles Sunset Lodge
6,Bengkulu,ID,84.56,few clouds,-3.8004,102.2655,Grage Hotel Bengkulu
9,Naze,JP,83.32,scattered clouds,28.3667,129.4833,オーベルジュ・アガルイ
10,Big Spring,US,81.88,clear sky,32.2504,-101.4787,Hampton Inn & Suites Big Spring
13,Hilo,US,78.53,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
15,Victoria,HK,84.38,overcast clouds,22.2855,114.1577,Mini Hotel Central
18,Avarua,CK,77.05,scattered clouds,-21.2078,-159.7750,Paradise Inn
28,Puerto Escondido,MX,80.40,overcast clouds,15.8500,-97.0667,Aldea del Bazar Hotel and Spa
39,Porbandar,IN,80.58,haze,21.6422,69.6093,Lords Inn Porbandar
40,Arlit,NE,80.13,clear sky,18.7369,7.3853,Case de Passage De la Prefecture


In [26]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   174
Country                174
Max Temp               174
Current Description    174
Lat                    174
Lng                    174
Hotel Name             174
dtype: int64

In [27]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [37]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt>{Current Description} and {Max Temp} °F</dd>
"""

In [38]:
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [40]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

In [41]:
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))